# What Makes an NBA Champion?

#### Daniel Abboudi and Sean Campi
##### Data Bootcamp, NYU Stern 4/25/2021

In [2]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
from sklearn.manifold import TSNE
import plotly.express as px

Data is from [NBA.com](https://www.nba.com/stats/teams/)
<br>
Includes Advanced Stats, Traditional Stats (standardized over 100 possessions), and Opponent Stats (standardized over 100 possessions)
<br>
<br>
*Note: Unfortunately, NBA.com does not allow for scraping or downloading of their data. We copy and pasted the relevant data into three excel files and uploaded them to the project's github repository*

In [8]:
# Import the data sets
adv = pd.read_excel('https://github.com/danielabboudi/DB_Project/raw/main/NBA_Advanced.xlsx')
trad = pd.read_excel('https://github.com/danielabboudi/DB_Project/raw/main/NBA_Traditional_per100.xlsx')
opp = pd.read_excel('https://github.com/danielabboudi/DB_Project/raw/main/NBA_Opponent_per100.xlsx')

In [10]:
adv.columns

Index(['Season', 'TEAM', 'GP', 'W', 'L', 'MIN', 'OFFRTG', 'DEFRTG', 'NETRTG',
       'AST%', 'AST/TO', 'AST RATIO', 'OREB%', 'DREB%', 'REB%', 'TOV%', 'EFG%',
       'TS%', 'PACE', 'PIE', 'POSS'],
      dtype='object')

In [22]:
# Merge the data sets
merge1 = pd.merge(adv,trad,how='left',left_on=['Season','TEAM'],right_on=['Season','TEAM'])
df = pd.merge(merge1,opp,how='left',left_on=['Season','TEAM'],right_on=['Season','TEAM'])
df

,Season,TEAM,GP_x,W_x,L_x,MIN_x,OFFRTG,DEFRTG,NETRTG,AST%,...,OPPREB,OPPAST,OPPTOV,OPPSTL,OPPBLK,OPPBLKA,OPPPF,OPPPFD,OPPPTS,OPP +/-
0,2001,San Antonio Spurs,82,58,24,3966,105.0,96.6,8.3,61.7,...,45.2,21.5,14.7,8.0,5.7,7.7,25.9,20.6,96.5,-8.5
1,2001,Los Angeles Lakers,82,56,26,3981,107.0,103.6,3.5,60.7,...,43.9,21.5,13.9,8.0,4.2,6.4,26.0,24.3,103.4,-3.6
2,2001,Philadelphia 76ers,82,56,26,3971,101.9,97.3,4.6,58.3,...,44.0,23.2,16.6,8.8,6.0,5.4,25.4,22.0,97.3,-4.6
3,2001,Sacramento Kings,82,55,27,4016,104.2,98.5,5.7,59.1,...,47.0,22.7,16.7,8.3,5.9,5.4,22.0,19.9,98.2,-6.0
4,2001,Dallas Mavericks,82,53,29,3961,105.6,101.2,4.4,56.4,...,47.3,22.8,16.3,8.0,4.9,6.3,23.7,24.4,101.1,-4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,2021,Oklahoma City Thunder,60,20,40,2895,103.6,112.8,-9.2,57.2,...,46.2,24.9,12.8,8.9,5.0,4.4,18.0,17.8,112.6,9.3
622,2021,Detroit Pistons,61,18,43,2958,108.2,112.3,-4.1,63.0,...,44.6,24.8,14.4,8.0,5.9,5.2,20.8,20.9,111.9,3.8
623,2021,Orlando Magic,59,18,41,2842,105.0,113.3,-8.3,57.5,...,47.2,26.2,13.2,7.6,5.3,4.3,18.6,17.0,113.7,8.5
624,2021,Minnesota Timberwolves,61,17,44,2948,108.0,114.9,-6.9,62.2,...,45.0,26.1,14.8,7.5,5.5,5.4,19.2,20.3,115.0,6.9


In [23]:
# Select the columns of data we want to use
df.columns
remove_col = ['GP_x','W_x','L_x','MIN_x','AST RATIO','EFG%','PIE','POSS','GP_y','W_y','L_y','MIN_y','PTS','FGM','3PM',
             'FTM','OREB','DREB','REB','AST','TOV','BLKA','PF','PFD','+/-','Unnamed: 28','GP','W','L','MIN','OPPFGM',
             'OPP3PM','OPPFTM','OPPOREB','OPPDREB','OPPREB', 'OPPBLKA', 'OPPPF', 'OPPPFD', 'OPPPTS', 'OPP +/-']

df.drop(remove_col,axis=1)

,Season,TEAM,OFFRTG,DEFRTG,NETRTG,AST%,AST/TO,OREB%,DREB%,REB%,...,OPPFGA,OPPFG%,OPP3PA,OPP3P%,OPPFTA,OPPFT%,OPPAST,OPPTOV,OPPSTL,OPPBLK
0,2001,San Antonio Spurs,105.0,96.6,8.3,61.7,1.55,29.7,70.3,51.1,...,90.1,41.9,13.9,32.9,22.1,74.1,21.5,14.7,8.0,5.7
1,2001,Los Angeles Lakers,107.0,103.6,3.5,60.7,1.59,33.8,69.1,51.7,...,88.4,43.8,15.1,35.4,27.4,75.4,21.5,13.9,8.0,4.2
2,2001,Philadelphia 76ers,101.9,97.3,4.6,58.3,1.31,33.4,70.4,52.3,...,87.9,42.9,16.4,34.2,21.8,74.9,23.2,16.6,8.8,6.0
3,2001,Sacramento Kings,104.2,98.5,5.7,59.1,1.52,30.7,67.9,49.7,...,89.1,43.2,14.1,35.5,21.5,74.9,22.7,16.7,8.3,5.9
4,2001,Dallas Mavericks,105.6,101.2,4.4,56.4,1.52,27.2,67.8,48.1,...,85.8,43.9,14.6,33.4,28.6,73.1,22.8,16.3,8.0,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,2021,Oklahoma City Thunder,103.6,112.8,-9.2,57.2,1.40,24.7,74.0,49.3,...,90.1,47.0,36.1,36.2,19.3,76.7,24.9,12.8,8.9,5.0
622,2021,Detroit Pistons,108.2,112.3,-4.1,63.0,1.64,27.4,72.4,49.5,...,87.4,47.6,31.3,36.6,22.7,76.3,24.8,14.4,8.0,5.9
623,2021,Orlando Magic,105.0,113.3,-8.3,57.5,1.73,25.1,74.9,48.9,...,89.4,47.4,37.0,37.6,19.0,79.1,26.2,13.2,7.6,5.3
624,2021,Minnesota Timberwolves,108.0,114.9,-6.9,62.2,1.75,26.7,71.5,48.1,...,87.2,48.2,32.6,39.4,22.9,79.2,26.1,14.8,7.5,5.5
